In [1]:
import time
import random
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from google.colab import output

class TicTacToe:
    def __init__(self):
        # Initialize the 3x3 board with empty spaces
        self.board = [' ' for _ in range(9)]
        self.current_winner = None

    def print_board(self):
        """Print the current state of the board"""
        for row in [self.board[i*3:(i+1)*3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    def print_board_nums(self):
        """Print the board with position numbers for reference"""
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def make_move(self, square, letter):
        """Make a move on the board"""
        if self.board[square] == ' ':
            self.board[square] = letter
            # Check if this move results in a win
            if self.check_winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def check_winner(self, square, letter):
        """Check if the current move results in a win"""
        # Check row
        row_ind = square // 3
        row = self.board[row_ind*3:(row_ind+1)*3]
        if all([spot == letter for spot in row]):
            return True

        # Check column
        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True

        # Check diagonals
        # Only if the square is on a diagonal
        if square % 2 == 0:
            # Check main diagonal (top-left to bottom-right)
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            if all([spot == letter for spot in diagonal1]):
                return True
            # Check other diagonal (top-right to bottom-left)
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            if all([spot == letter for spot in diagonal2]):
                return True

        # No win
        return False

    def available_moves(self):
        """Return a list of available moves"""
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def empty_squares(self):
        """Check if there are empty squares on the board"""
        return ' ' in self.board

    def num_empty_squares(self):
        """Return the number of empty squares"""
        return self.board.count(' ')

    def is_board_full(self):
        """Check if the board is full"""
        return not self.empty_squares()

    def clone(self):
        """Create a deep copy of the current game state"""
        new_game = TicTacToe()
        new_game.board = self.board.copy()
        new_game.current_winner = self.current_winner
        return new_game

class Player:
    def __init__(self, letter):
        # Letter is 'X' or 'O'
        self.letter = letter

    def get_move(self, game):
        """Get the player's move"""
        pass

class MinimaxPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)
        # Track the number of states evaluated
        self.states_evaluated = 0

    def get_move(self, game):
        """Get the best move using the minimax algorithm"""
        self.states_evaluated = 0

        if len(game.available_moves()) == 9:
            # First move optimization - just pick a corner or center
            square = random.choice([0, 2, 4, 6, 8])
        else:
            # Reset states evaluated counter
            start_time = time.time()

            # Get best move using minimax
            square = self.minimax(game, self.letter)['position']

            end_time = time.time()
            print(f"Minimax evaluated {self.states_evaluated} states in {end_time - start_time:.5f} seconds")

        return square

    def minimax(self, state, player):
        """Standard minimax algorithm for tic-tac-toe"""
        self.states_evaluated += 1

        # Opponent letter
        other_player = 'O' if player == 'X' else 'X'

        # Check if previous move is a winner
        if state.current_winner == other_player:
            # If opponent won, return position and negative score
            return {
                'position': None,
                'score': 1 * (state.num_empty_squares() + 1) if other_player == self.letter else -1 * (state.num_empty_squares() + 1)
            }

        # If no empty squares, it's a tie
        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        # Initialize the best move
        if player == self.letter:
            # If current player is the AI, we want to maximize score
            best = {'position': None, 'score': float('-inf')}
        else:
            # If current player is the opponent, we want to minimize score
            best = {'position': None, 'score': float('inf')}

        # Try each available move and evaluate
        for possible_move in state.available_moves():
            # Make the move
            state_copy = state.clone()
            state_copy.make_move(possible_move, player)

            # Recursively simulate the game after this move
            sim_score = self.minimax(state_copy, other_player)

            # Undo the move (implicit by using a copy)
            sim_score['position'] = possible_move  # This is the move that led to this score

            # Update best move if better than current best
            if player == self.letter:
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score

        return best

class AlphaBetaPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)
        # Track the number of states evaluated
        self.states_evaluated = 0

    def get_move(self, game):
        """Get the best move using the alpha-beta pruning algorithm"""
        self.states_evaluated = 0

        if len(game.available_moves()) == 9:
            # First move optimization - just pick a corner or center
            square = random.choice([0, 2, 4, 6, 8])
        else:
            # Reset states evaluated counter
            start_time = time.time()

            # Get best move using alpha-beta pruning
            square = self.alpha_beta(game, self.letter, float('-inf'), float('inf'))['position']

            end_time = time.time()
            print(f"Alpha-Beta Pruning evaluated {self.states_evaluated} states in {end_time - start_time:.5f} seconds")

        return square

    def alpha_beta(self, state, player, alpha, beta):
        """Alpha-beta pruning optimized minimax algorithm"""
        self.states_evaluated += 1

        # Opponent letter
        other_player = 'O' if player == 'X' else 'X'

        # Check if previous move is a winner
        if state.current_winner == other_player:
            # If opponent won, return position and negative score
            return {
                'position': None,
                'score': 1 * (state.num_empty_squares() + 1) if other_player == self.letter else -1 * (state.num_empty_squares() + 1)
            }

        # If no empty squares, it's a tie
        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        # Initialize the best move
        if player == self.letter:
            # If current player is the AI, we want to maximize score
            best = {'position': None, 'score': float('-inf')}
        else:
            # If current player is the opponent, we want to minimize score
            best = {'position': None, 'score': float('inf')}

        # Try each available move and evaluate
        for possible_move in state.available_moves():
            # Make the move
            state_copy = state.clone()
            state_copy.make_move(possible_move, player)

            # Recursively simulate the game after this move
            sim_score = self.alpha_beta(state_copy, other_player, alpha, beta)

            # Undo the move (implicit by using a copy)
            sim_score['position'] = possible_move  # This is the move that led to this score

            # Update best move if better than current best
            if player == self.letter:
                if sim_score['score'] > best['score']:
                    best = sim_score
                # Update alpha
                alpha = max(alpha, best['score'])
                if beta <= alpha:
                    break  # Beta cutoff
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score
                # Update beta
                beta = min(beta, best['score'])
                if beta <= alpha:
                    break  # Alpha cutoff

        return best

class RandomComputerPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)

    def get_move(self, game):
        """Pick a random valid spot for next move"""
        square = random.choice(game.available_moves())
        return square

class VisualTicTacToe:
    def __init__(self):
        self.game = TicTacToe()
        self.current_letter = 'X'
        self.players = {'X': None, 'O': None}
        self.status_label = widgets.Label(value="Welcome to Tic-Tac-Toe!")
        self.game_in_progress = False
        self.game_over = False
        self.ai_thinking = False

        # Create buttons for the game board
        self.buttons = []
        for i in range(9):
            btn = widgets.Button(
                description='',
                disabled=True,
                button_style='',
                layout=widgets.Layout(width='80px', height='80px', font_size='24px')
            )
            btn.add_class(f'square-{i}')
            btn.on_click(self.make_human_move)
            self.buttons.append(btn)

        # Create the grid layout for the game board
        self.board_grid = widgets.GridBox(
            children=self.buttons,
            layout=widgets.Layout(
                grid_template_columns='repeat(3, 80px)',
                grid_template_rows='repeat(3, 80px)',
                grid_gap='2px 2px',
                width='250px',
                height='250px'
            )
        )

        # Create buttons for game mode selection
        self.game_modes = [
            ('Human vs Minimax AI', self.setup_human_vs_minimax),
            ('Human vs Alpha-Beta AI', self.setup_human_vs_alphabeta),
            ('Minimax AI vs Alpha-Beta AI', self.setup_minimax_vs_alphabeta),
            ('Random AI vs Minimax AI', self.setup_random_vs_minimax),
            ('Random AI vs Alpha-Beta AI', self.setup_random_vs_alphabeta)
        ]

        self.mode_buttons = []
        for mode, _ in self.game_modes:
            btn = widgets.Button(
                description=mode,
                layout=widgets.Layout(width='200px')
            )
            self.mode_buttons.append(btn)

        # Create buttons for player letter selection
        self.letter_selection = widgets.RadioButtons(
            options=['X (goes first)', 'O (goes second)'],
            value='X (goes first)',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )

        # Create start game button
        self.start_button = widgets.Button(
            description='Start Game',
            button_style='success',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )
        self.start_button.on_click(self.start_game)

        # Create reset game button
        self.reset_button = widgets.Button(
            description='Reset Game',
            button_style='danger',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )
        self.reset_button.on_click(self.reset_game)

        # Attach event handlers to mode buttons
        for i, (_, handler) in enumerate(self.game_modes):
            self.mode_buttons[i].on_click(handler)

        # Create the main layout
        self.header = widgets.HTML(value="<h2>Tic-Tac-Toe with AI</h2>")
        self.main_layout = widgets.VBox([
            self.header,
            widgets.HBox([
                widgets.VBox([
                    widgets.HTML(value="<h3>Game Modes:</h3>"),
                    *self.mode_buttons,
                    widgets.HTML(value="<h3>Choose Your Letter:</h3>"),
                    self.letter_selection,
                    self.start_button,
                    self.reset_button,
                    self.status_label
                ]),
                widgets.VBox([
                    widgets.HTML(value="<h3>Game Board:</h3>"),
                    self.board_grid
                ])
            ])
        ])

        # Apply some basic styling
        display(HTML("""
        <style>
        .square-0, .square-1, .square-2, .square-3, .square-4, .square-5, .square-6, .square-7, .square-8 {
            font-size: 28px !important;
            font-weight: bold !important;
        }
        </style>
        """))

    def setup_human_vs_minimax(self, btn):
        self.reset_game_state()
        self.letter_selection.disabled = False
        self.start_button.disabled = False
        self.game_mode = 'human_vs_minimax'
        self.status_label.value = "Choose your letter and click Start Game"

        # Highlight the selected mode button
        for b in self.mode_buttons:
            b.button_style = ''
        btn.button_style = 'info'

    def setup_human_vs_alphabeta(self, btn):
        self.reset_game_state()
        self.letter_selection.disabled = False
        self.start_button.disabled = False
        self.game_mode = 'human_vs_alphabeta'
        self.status_label.value = "Choose your letter and click Start Game"

        # Highlight the selected mode button
        for b in self.mode_buttons:
            b.button_style = ''
        btn.button_style = 'info'

    def setup_minimax_vs_alphabeta(self, btn):
        self.reset_game_state()
        self.letter_selection.disabled = True
        self.start_button.disabled = False
        self.game_mode = 'minimax_vs_alphabeta'
        self.status_label.value = "Click Start Game to watch Minimax vs Alpha-Beta"

        # Highlight the selected mode button
        for b in self.mode_buttons:
            b.button_style = ''
        btn.button_style = 'info'

    def setup_random_vs_minimax(self, btn):
        self.reset_game_state()
        self.letter_selection.disabled = True
        self.start_button.disabled = False
        self.game_mode = 'random_vs_minimax'
        self.status_label.value = "Click Start Game to watch Random vs Minimax"

        # Highlight the selected mode button
        for b in self.mode_buttons:
            b.button_style = ''
        btn.button_style = 'info'

    def setup_random_vs_alphabeta(self, btn):
        self.reset_game_state()
        self.letter_selection.disabled = True
        self.start_button.disabled = False
        self.game_mode = 'random_vs_alphabeta'
        self.status_label.value = "Click Start Game to watch Random vs Alpha-Beta"

        # Highlight the selected mode button
        for b in self.mode_buttons:
            b.button_style = ''
        btn.button_style = 'info'

    def start_game(self, btn):
        self.game_in_progress = True
        self.game_over = False
        self.reset_button.disabled = False

        # Set up players based on game mode
        if self.game_mode == 'human_vs_minimax':
            human_letter = 'X' if 'X' in self.letter_selection.value else 'O'
            ai_letter = 'O' if human_letter == 'X' else 'X'
            self.players = {
                human_letter: 'human',
                ai_letter: MinimaxPlayer(ai_letter)
            }

        elif self.game_mode == 'human_vs_alphabeta':
            human_letter = 'X' if 'X' in self.letter_selection.value else 'O'
            ai_letter = 'O' if human_letter == 'X' else 'X'
            self.players = {
                human_letter: 'human',
                ai_letter: AlphaBetaPlayer(ai_letter)
            }

        elif self.game_mode == 'minimax_vs_alphabeta':
            self.players = {
                'X': MinimaxPlayer('X'),
                'O': AlphaBetaPlayer('O')
            }

        elif self.game_mode == 'random_vs_minimax':
            self.players = {
                'X': RandomComputerPlayer('X'),
                'O': MinimaxPlayer('O')
            }

        elif self.game_mode == 'random_vs_alphabeta':
            self.players = {
                'X': RandomComputerPlayer('X'),
                'O': AlphaBetaPlayer('O')
            }

        # Enable game board buttons if human is playing
        for i in range(9):
            self.buttons[i].disabled = False
            self.buttons[i].description = ''
            self.buttons[i].button_style = ''

        self.status_label.value = f"Game started! {self.current_letter}'s turn"

        # Disable mode selection and start button during game
        for btn in self.mode_buttons:
            btn.disabled = True
        self.letter_selection.disabled = True
        self.start_button.disabled = True

        # Make AI move if it goes first
        self.check_and_make_ai_move()

    def reset_game(self, btn):
        self.reset_game_state()

        # Re-enable mode selection
        for btn in self.mode_buttons:
            btn.disabled = False
            btn.button_style = ''

        self.letter_selection.disabled = True
        self.start_button.disabled = True
        self.reset_button.disabled = True
        self.status_label.value = "Select a game mode to begin"

    def reset_game_state(self):
        self.game = TicTacToe()
        self.current_letter = 'X'
        self.game_in_progress = False
        self.game_over = False

        # Reset all buttons
        for i in range(9):
            self.buttons[i].description = ''
            self.buttons[i].disabled = True
            self.buttons[i].button_style = ''

    def make_human_move(self, btn):
        if self.game_over or self.ai_thinking or self.players[self.current_letter] != 'human':
            return

        # Find which button was clicked
        square = self.buttons.index(btn)

        # Make move if valid
        if self.game.board[square] == ' ':
            self.make_move(square)

    def make_move(self, square):
        if self.game_over:
            return

        # Make the move and update the display
        if self.game.make_move(square, self.current_letter):
            # Update button display
            self.buttons[square].description = self.current_letter

            # Set button style based on player
            if self.current_letter == 'X':
                self.buttons[square].button_style = 'danger'
            else:
                self.buttons[square].button_style = 'info'

            # Check for winner
            if self.game.current_winner:
                self.status_label.value = f"{self.current_letter} wins!"
                self.game_over = True
                self.disable_board()
                return

            # Check for tie
            if self.game.is_board_full():
                self.status_label.value = "It's a tie!"
                self.game_over = True
                self.disable_board()
                return

            # Switch player
            self.current_letter = 'O' if self.current_letter == 'X' else 'X'
            self.status_label.value = f"{self.current_letter}'s turn"

            # Check if next player is AI
            self.check_and_make_ai_move()

    def check_and_make_ai_move(self):
        """Check if current player is AI and make a move if so"""
        if self.game_over:
            return

        current_player = self.players[self.current_letter]
        if current_player != 'human':
            # Signal AI is thinking
            self.ai_thinking = True
            self.status_label.value = f"{self.current_letter} (AI) is thinking..."

            # Add a small delay to make the game more natural
            import threading

            def ai_move():
                time.sleep(0.5)  # Small delay to show AI is "thinking"
                # Get AI move
                square = current_player.get_move(self.game)

                # Update UI in the main thread
                import IPython
                IPython.display.display(IPython.display.Javascript('''
                    setTimeout(function() {
                        IPython.notebook.kernel.execute("visual_game.make_move(" + arguments[0] + ")");
                    }, 100);
                ''', square))

                self.ai_thinking = False

            # Start AI move in separate thread
            threading.Thread(target=ai_move).start()

    def disable_board(self):
        """Disable all board buttons at game end"""
        for btn in self.buttons:
            btn.disabled = True

        # Re-enable mode selection
        for btn in self.mode_buttons:
            btn.disabled = False

    def display(self):
        """Display the game UI"""
        display(self.main_layout)

def run_visual_game():
    """Run the visual Tic-Tac-Toe game"""
    global visual_game
    visual_game = VisualTicTacToe()
    visual_game.display()
# Run the game
run_visual_game()

In [2]:
import time
import random
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from google.colab import output
from functools import lru_cache

class TicTacToe:
    def __init__(self):
        # Initialize the 3x3 board with empty spaces
        self.board = [' ' for _ in range(9)]
        self.current_winner = None

    def print_board(self):
        """Print the current state of the board"""
        for row in [self.board[i*3:(i+1)*3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    def print_board_nums(self):
        """Print the board with position numbers for reference"""
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def make_move(self, square, letter):
        """Make a move on the board"""
        if self.board[square] == ' ':
            self.board[square] = letter
            # Check if this move results in a win
            if self.check_winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def check_winner(self, square, letter):
        """Check if the current move results in a win"""
        # Check row
        row_ind = square // 3
        row = self.board[row_ind*3:(row_ind+1)*3]
        if all([spot == letter for spot in row]):
            return True

        # Check column
        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True

        # Check diagonals
        # Only if the square is on a diagonal
        if square % 2 == 0:
            # Check main diagonal (top-left to bottom-right)
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            if all([spot == letter for spot in diagonal1]):
                return True
            # Check other diagonal (top-right to bottom-left)
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            if all([spot == letter for spot in diagonal2]):
                return True

        # No win
        return False

    def available_moves(self):
        """Return a list of available moves"""
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def empty_squares(self):
        """Check if there are empty squares on the board"""
        return ' ' in self.board

    def num_empty_squares(self):
        """Return the number of empty squares"""
        return self.board.count(' ')

    def is_board_full(self):
        """Check if the board is full"""
        return not self.empty_squares()

    def clone(self):
        """Create a deep copy of the current game state"""
        new_game = TicTacToe()
        new_game.board = self.board.copy()
        new_game.current_winner = self.current_winner
        return new_game

    def board_hash(self):
        """Create a hash of the current board state for caching"""
        return ''.join(self.board)

class Player:
    def __init__(self, letter):
        # Letter is 'X' or 'O'
        self.letter = letter

    def get_move(self, game):
        """Get the player's move"""
        pass

class MinimaxPlayer(Player):
    def __init__(self, letter, max_depth=None):
        super().__init__(letter)
        # Track the number of states evaluated
        self.states_evaluated = 0
        self.max_depth = max_depth  # Depth limit for early game

    def get_move(self, game):
        """Get the best move using the minimax algorithm"""
        self.states_evaluated = 0

        if len(game.available_moves()) == 9:
            # First move optimization - just pick a corner or center
            square = random.choice([0, 2, 4, 6, 8])
        else:
            # Reset states evaluated counter
            start_time = time.time()

            # Calculate appropriate depth based on game state
            current_depth = self.max_depth if self.max_depth is not None else float('inf')
            if self.max_depth is None:
                # Dynamic depth based on number of empty squares
                empty_squares = game.num_empty_squares()
                if empty_squares > 6:  # Early game
                    current_depth = 3
                elif empty_squares > 4:  # Mid game
                    current_depth = 5
                else:  # End game
                    current_depth = float('inf')

            # Get best move using minimax
            square = self.minimax(game, self.letter, current_depth)['position']

            end_time = time.time()
            print(f"Minimax evaluated {self.states_evaluated} states in {end_time - start_time:.5f} seconds")

        return square

    def minimax(self, state, player, depth):
        """Standard minimax algorithm for tic-tac-toe with depth limit"""
        self.states_evaluated += 1

        # Opponent letter
        other_player = 'O' if player == 'X' else 'X'

        # Check if previous move is a winner
        if state.current_winner == other_player:
            # If opponent won, return position and negative score
            return {
                'position': None,
                'score': 1 * (state.num_empty_squares() + 1) if other_player == self.letter else -1 * (state.num_empty_squares() + 1)
            }

        # If no empty squares, it's a tie
        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        # If we've reached depth limit, use heuristic evaluation
        if depth == 0:
            return {'position': None, 'score': self.heuristic_evaluation(state)}

        # Initialize the best move
        if player == self.letter:
            # If current player is the AI, we want to maximize score
            best = {'position': None, 'score': float('-inf')}
        else:
            # If current player is the opponent, we want to minimize score
            best = {'position': None, 'score': float('inf')}

        # Get available moves with better move ordering
        available_moves = self.ordered_moves(state)

        # Try each available move and evaluate
        for possible_move in available_moves:
            # Make the move
            state_copy = state.clone()
            state_copy.make_move(possible_move, player)

            # Recursively simulate the game after this move
            sim_score = self.minimax(state_copy, other_player, depth-1)

            # Undo the move (implicit by using a copy)
            sim_score['position'] = possible_move  # This is the move that led to this score

            # Update best move if better than current best
            if player == self.letter:
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score

        return best

    def ordered_moves(self, state):
        """Return moves in better order (center and corners first)"""
        moves = state.available_moves()
        # Prefer center, then corners, then edges
        move_priority = [4, 0, 2, 6, 8, 1, 3, 5, 7]
        ordered = []
        for move in move_priority:
            if move in moves:
                ordered.append(move)
        return ordered

    def heuristic_evaluation(self, state):
        """Simple heuristic evaluation for early game states"""
        # Count potential winning lines
        score = 0
        lines = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
            [0, 4, 8], [2, 4, 6]              # diagonals
        ]

        for line in lines:
            values = [state.board[i] for i in line]
            if values.count(self.letter) == 2 and values.count(' ') == 1:
                score += 10
            elif values.count(self.letter) == 1 and values.count(' ') == 2:
                score += 1
            opponent = 'O' if self.letter == 'X' else 'X'
            if values.count(opponent) == 2 and values.count(' ') == 1:
                score -= 10
            elif values.count(opponent) == 1 and values.count(' ') == 2:
                score -= 1

        return score

class AlphaBetaPlayer(Player):
    def __init__(self, letter, max_depth=None):
        super().__init__(letter)
        # Track the number of states evaluated
        self.states_evaluated = 0
        self.max_depth = max_depth  # Depth limit for early game
        self.cache = {}  # Cache for memoization

    def get_move(self, game):
        """Get the best move using the alpha-beta pruning algorithm"""
        self.states_evaluated = 0

        if len(game.available_moves()) == 9:
            # First move optimization - just pick a corner or center
            square = random.choice([0, 2, 4, 6, 8])
        else:
            # Reset states evaluated counter
            start_time = time.time()

            # Calculate appropriate depth based on game state
            current_depth = self.max_depth if self.max_depth is not None else float('inf')
            if self.max_depth is None:
                # Dynamic depth based on number of empty squares
                empty_squares = game.num_empty_squares()
                if empty_squares > 6:  # Early game
                    current_depth = 3
                elif empty_squares > 4:  # Mid game
                    current_depth = 5
                else:  # End game
                    current_depth = float('inf')

            # Get best move using alpha-beta pruning
            square = self.alpha_beta(game, self.letter, float('-inf'), float('inf'), current_depth)['position']

            end_time = time.time()
            print(f"Alpha-Beta Pruning evaluated {self.states_evaluated} states in {end_time - start_time:.5f} seconds")

        return square

    def alpha_beta(self, state, player, alpha, beta, depth):
        """Alpha-beta pruning optimized minimax algorithm with depth limit and caching"""
        self.states_evaluated += 1

        # Check cache first
        cache_key = (state.board_hash(), player, alpha, beta, depth)
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Opponent letter
        other_player = 'O' if player == 'X' else 'X'

        # Check if previous move is a winner
        if state.current_winner == other_player:
            # If opponent won, return position and negative score
            result = {
                'position': None,
                'score': 1 * (state.num_empty_squares() + 1) if other_player == self.letter else -1 * (state.num_empty_squares() + 1)
            }
            self.cache[cache_key] = result
            return result

        # If no empty squares, it's a tie
        elif not state.empty_squares():
            result = {'position': None, 'score': 0}
            self.cache[cache_key] = result
            return result

        # If we've reached depth limit, use heuristic evaluation
        if depth == 0:
            result = {'position': None, 'score': self.heuristic_evaluation(state)}
            self.cache[cache_key] = result
            return result

        # Initialize the best move
        if player == self.letter:
            # If current player is the AI, we want to maximize score
            best = {'position': None, 'score': float('-inf')}
        else:
            # If current player is the opponent, we want to minimize score
            best = {'position': None, 'score': float('inf')}

        # Get available moves with better move ordering
        available_moves = self.ordered_moves(state)

        # Try each available move and evaluate
        for possible_move in available_moves:
            # Make the move
            state_copy = state.clone()
            state_copy.make_move(possible_move, player)

            # Recursively simulate the game after this move
            sim_score = self.alpha_beta(state_copy, other_player, alpha, beta, depth-1)

            # Undo the move (implicit by using a copy)
            sim_score['position'] = possible_move  # This is the move that led to this score

            # Update best move if better than current best
            if player == self.letter:
                if sim_score['score'] > best['score']:
                    best = sim_score
                # Update alpha
                alpha = max(alpha, best['score'])
                if beta <= alpha:
                    break  # Beta cutoff
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score
                # Update beta
                beta = min(beta, best['score'])
                if beta <= alpha:
                    break  # Alpha cutoff

        self.cache[cache_key] = best
        return best

    def ordered_moves(self, state):
        """Return moves in better order (center and corners first)"""
        moves = state.available_moves()
        # Prefer center, then corners, then edges
        move_priority = [4, 0, 2, 6, 8, 1, 3, 5, 7]
        ordered = []
        for move in move_priority:
            if move in moves:
                ordered.append(move)
        return ordered

    def heuristic_evaluation(self, state):
        """Simple heuristic evaluation for early game states"""
        # Count potential winning lines
        score = 0
        lines = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
            [0, 4, 8], [2, 4, 6]              # diagonals
        ]

        for line in lines:
            values = [state.board[i] for i in line]
            if values.count(self.letter) == 2 and values.count(' ') == 1:
                score += 10
            elif values.count(self.letter) == 1 and values.count(' ') == 2:
                score += 1
            opponent = 'O' if self.letter == 'X' else 'X'
            if values.count(opponent) == 2 and values.count(' ') == 1:
                score -= 10
            elif values.count(opponent) == 1 and values.count(' ') == 2:
                score -= 1

        return score

class RandomComputerPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)

    def get_move(self, game):
        """Pick a random valid spot for next move"""
        square = random.choice(game.available_moves())
        return square

class VisualTicTacToe:
    def __init__(self):
        self.game = TicTacToe()
        self.current_letter = 'X'
        self.players = {'X': None, 'O': None}
        self.status_label = widgets.Label(value="Welcome to Tic-Tac-Toe!")
        self.game_in_progress = False
        self.game_over = False
        self.ai_thinking = False

        # Create buttons for the game board
        self.buttons = []
        for i in range(9):
            btn = widgets.Button(
                description='',
                disabled=True,
                button_style='',
                layout=widgets.Layout(width='80px', height='80px', font_size='24px')
            )
            btn.add_class(f'square-{i}')
            btn.on_click(self.make_human_move)
            self.buttons.append(btn)

        # Create the grid layout for the game board
        self.board_grid = widgets.GridBox(
            children=self.buttons,
            layout=widgets.Layout(
                grid_template_columns='repeat(3, 80px)',
                grid_template_rows='repeat(3, 80px)',
                grid_gap='2px 2px',
                width='250px',
                height='250px'
            )
        )

        # Create buttons for game mode selection
        self.game_modes = [
            ('Human vs Minimax AI', self.setup_human_vs_minimax),
            ('Human vs Alpha-Beta AI', self.setup_human_vs_alphabeta),
            ('Minimax AI vs Alpha-Beta AI', self.setup_minimax_vs_alphabeta),
            ('Random AI vs Minimax AI', self.setup_random_vs_minimax),
            ('Random AI vs Alpha-Beta AI', self.setup_random_vs_alphabeta)
        ]

        self.mode_buttons = []
        for mode, _ in self.game_modes:
            btn = widgets.Button(
                description=mode,
                layout=widgets.Layout(width='200px')
            )
            self.mode_buttons.append(btn)

        # Create buttons for player letter selection
        self.letter_selection = widgets.RadioButtons(
            options=['X (goes first)', 'O (goes second)'],
            value='X (goes first)',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )

        # Create start game button
        self.start_button = widgets.Button(
            description='Start Game',
            button_style='success',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )
        self.start_button.on_click(self.start_game)

        # Create reset game button
        self.reset_button = widgets.Button(
            description='Reset Game',
            button_style='danger',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )
        self.reset_button.on_click(self.reset_game)

        # Attach event handlers to mode buttons
        for i, (_, handler) in enumerate(self.game_modes):
            self.mode_buttons[i].on_click(handler)

        # Create the main layout
        self.header = widgets.HTML(value="<h2>Tic-Tac-Toe with AI</h2>")
        self.main_layout = widgets.VBox([
            self.header,
            widgets.HBox([
                widgets.VBox([
                    widgets.HTML(value="<h3>Game Modes:</h3>"),
                    *self.mode_buttons,
                    widgets.HTML(value="<h3>Choose Your Letter:</h3>"),
                    self.letter_selection,
                    self.start_button,
                    self.reset_button,
                    self.status_label
                ]),
                widgets.VBox([
                    widgets.HTML(value="<h3>Game Board:</h3>"),
                    self.board_grid
                ])
            ])
        ])

        # Apply some basic styling
        display(HTML("""
        <style>
        .square-0, .square-1, .square-2, .square-3, .square-4, .square-5, .square-6, .square-7, .square-8 {
            font-size: 28px !important;
            font-weight: bold !important;
        }
        </style>
        """))

    def setup_human_vs_minimax(self, btn):
        self.reset_game_state()
        self.letter_selection.disabled = False
        self.start_button.disabled = False
        self.game_mode = 'human_vs_minimax'
        self.status_label.value = "Choose your letter and click Start Game"

        # Highlight the selected mode button
        for b in self.mode_buttons:
            b.button_style = ''
        btn.button_style = 'info'

    def setup_human_vs_alphabeta(self, btn):
        self.reset_game_state()
        self.letter_selection.disabled = False
        self.start_button.disabled = False
        self.game_mode = 'human_vs_alphabeta'
        self.status_label.value = "Choose your letter and click Start Game"

        # Highlight the selected mode button
        for b in self.mode_buttons:
            b.button_style = ''
        btn.button_style = 'info'

    def setup_minimax_vs_alphabeta(self, btn):
        self.reset_game_state()
        self.letter_selection.disabled = True
        self.start_button.disabled = False
        self.game_mode = 'minimax_vs_alphabeta'
        self.status_label.value = "Click Start Game to watch Minimax vs Alpha-Beta"

        # Highlight the selected mode button
        for b in self.mode_buttons:
            b.button_style = ''
        btn.button_style = 'info'

    def setup_random_vs_minimax(self, btn):
        self.reset_game_state()
        self.letter_selection.disabled = True
        self.start_button.disabled = False
        self.game_mode = 'random_vs_minimax'
        self.status_label.value = "Click Start Game to watch Random vs Minimax"

        # Highlight the selected mode button
        for b in self.mode_buttons:
            b.button_style = ''
        btn.button_style = 'info'

    def setup_random_vs_alphabeta(self, btn):
        self.reset_game_state()
        self.letter_selection.disabled = True
        self.start_button.disabled = False
        self.game_mode = 'random_vs_alphabeta'
        self.status_label.value = "Click Start Game to watch Random vs Alpha-Beta"

        # Highlight the selected mode button
        for b in self.mode_buttons:
            b.button_style = ''
        btn.button_style = 'info'

    def start_game(self, btn):
        self.game_in_progress = True
        self.game_over = False
        self.reset_button.disabled = False

        # Set up players based on game mode
        if self.game_mode == 'human_vs_minimax':
            human_letter = 'X' if 'X' in self.letter_selection.value else 'O'
            ai_letter = 'O' if human_letter == 'X' else 'X'
            self.players = {
                human_letter: 'human',
                ai_letter: MinimaxPlayer(ai_letter)
            }

        elif self.game_mode == 'human_vs_alphabeta':
            human_letter = 'X' if 'X' in self.letter_selection.value else 'O'
            ai_letter = 'O' if human_letter == 'X' else 'X'
            self.players = {
                human_letter: 'human',
                ai_letter: AlphaBetaPlayer(ai_letter)
            }

        elif self.game_mode == 'minimax_vs_alphabeta':
            self.players = {
                'X': MinimaxPlayer('X'),
                'O': AlphaBetaPlayer('O')
            }

        elif self.game_mode == 'random_vs_minimax':
            self.players = {
                'X': RandomComputerPlayer('X'),
                'O': MinimaxPlayer('O')
            }

        elif self.game_mode == 'random_vs_alphabeta':
            self.players = {
                'X': RandomComputerPlayer('X'),
                'O': AlphaBetaPlayer('O')
            }

        # Enable game board buttons if human is playing
        for i in range(9):
            self.buttons[i].disabled = False
            self.buttons[i].description = ''
            self.buttons[i].button_style = ''

        self.status_label.value = f"Game started! {self.current_letter}'s turn"

        # Disable mode selection and start button during game
        for btn in self.mode_buttons:
            btn.disabled = True
        self.letter_selection.disabled = True
        self.start_button.disabled = True

        # Make AI move if it goes first
        self.check_and_make_ai_move()

    def reset_game(self, btn):
        self.reset_game_state()

        # Re-enable mode selection
        for btn in self.mode_buttons:
            btn.disabled = False
            btn.button_style = ''

        self.letter_selection.disabled = True
        self.start_button.disabled = True
        self.reset_button.disabled = True
        self.status_label.value = "Select a game mode to begin"

    def reset_game_state(self):
        self.game = TicTacToe()
        self.current_letter = 'X'
        self.game_in_progress = False
        self.game_over = False

        # Reset all buttons
        for i in range(9):
            self.buttons[i].description = ''
            self.buttons[i].disabled = True
            self.buttons[i].button_style = ''

    def make_human_move(self, btn):
        if self.game_over or self.ai_thinking or self.players[self.current_letter] != 'human':
            return

        # Find which button was clicked
        square = self.buttons.index(btn)

        # Make move if valid
        if self.game.board[square] == ' ':
            self.make_move(square)

    def make_move(self, square):
        if self.game_over:
            return

        # Make the move and update the display
        if self.game.make_move(square, self.current_letter):
            # Update button display
            self.buttons[square].description = self.current_letter

            # Set button style based on player
            if self.current_letter == 'X':
                self.buttons[square].button_style = 'danger'
            else:
                self.buttons[square].button_style = 'info'

            # Check for winner
            if self.game.current_winner:
                self.status_label.value = f"{self.current_letter} wins!"
                self.game_over = True
                self.disable_board()
                return

            # Check for tie
            if self.game.is_board_full():
                self.status_label.value = "It's a tie!"
                self.game_over = True
                self.disable_board()
                return

            # Switch player
            self.current_letter = 'O' if self.current_letter == 'X' else 'X'
            self.status_label.value = f"{self.current_letter}'s turn"

            # Check if next player is AI
            self.check_and_make_ai_move()

    def check_and_make_ai_move(self):
      """Check if current player is AI and make a move if so"""
      if self.game_over:
          return

      current_player = self.players[self.current_letter]
      if current_player != 'human':
          # Signal AI is thinking
          self.ai_thinking = True
          self.status_label.value = f"{self.current_letter} (AI) is thinking..."

          # Get AI move synchronously
          square = current_player.get_move(self.game)

          # Make the move
          self.make_move(square)

          self.ai_thinking = False

    def disable_board(self):
        """Disable all board buttons at game end"""
        for btn in self.buttons:
            btn.disabled = True

        # Re-enable mode selection
        for btn in self.mode_buttons:
            btn.disabled = False

    def display(self):
        """Display the game UI"""
        display(self.main_layout)

def run_visual_game():
    """Run the visual Tic-Tac-Toe game"""
    global visual_game
    visual_game = VisualTicTacToe()
    visual_game.display()

# Run the game
run_visual_game()

Minimax evaluated 401 states in 0.01117 seconds
Minimax evaluated 717 states in 0.00535 seconds
Minimax evaluated 51 states in 0.00041 seconds
Minimax evaluated 5 states in 0.00024 seconds
Alpha-Beta Pruning evaluated 154 states in 0.00357 seconds
Alpha-Beta Pruning evaluated 365 states in 0.00632 seconds
Alpha-Beta Pruning evaluated 47 states in 0.00039 seconds
Alpha-Beta Pruning evaluated 5 states in 0.00015 seconds
Alpha-Beta Pruning evaluated 83 states in 0.00137 seconds
Minimax evaluated 260 states in 0.00451 seconds
Alpha-Beta Pruning evaluated 253 states in 0.00406 seconds
Minimax evaluated 174 states in 0.00144 seconds
Alpha-Beta Pruning evaluated 28 states in 0.00034 seconds
Minimax evaluated 14 states in 0.00017 seconds
Alpha-Beta Pruning evaluated 4 states in 0.00006 seconds
Minimax evaluated 2 states in 0.00003 seconds
Minimax evaluated 401 states in 0.01202 seconds
Minimax evaluated 675 states in 0.01088 seconds
Minimax evaluated 34 states in 0.00031 seconds
Alpha-Beta Pru